## Lab 4: Previsão de Eleição de Deputados 

Autora: Lília Sampaio

Neste trabalho queremos prever através do uso de modelos de classificação quais candidatos a deputado foram eleitos em 2014. Primeiro, importamos as bibliotecas necessárias para fazer uma análise descritiva dos dados e conhecer suas propriedades. 

In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

### 1. Análise descritiva dos dados

Nesta atividade usaremos os dados referentes às eleições para deputado dos anos de 2006 à 2010 para classificar quais candidatos foram eleitos em 2014. Usaremos os dados de 2006 à 2010 como treino, e uma parte dos dados de 2014 como teste. Estes dados são carregados abaixo:

In [154]:
train = pd.read_csv("train.csv")

Um trecho inicial desses datasets nos mostra a natureza dos dados e as variáveis que encontramos à disposição para serem usadas na classificação, entre elas estado e partido do candidato, valor total de sua receita, despesas e recursos recebidos:

In [155]:
train.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.67,0.0,...,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,nao_eleito
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.85,6625.0,...,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,nao_eleito
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,158120.80,9301.22,2250.0,...,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,eleito
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.19,0.0,...,8,8,3001.12,375.14,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,nao_eleito
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,119820.00,2496.25,0.0,...,133,120,116416.64,875.31,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,eleito


Uma análise nos valores médios das variáveis, suas medianas, variância e distribuição podem ser vistos abaixo:

In [156]:
train.describe()

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa
count,7622.000000,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000
mean,2008.120703,9.039623e+10,27.673839,25.239963,1.732199e+05,5456.131027,1.965750e+04,2.315820e+04,7.921689e+04,2.577600e+04,2.541136e+04,130.485174,107.849777,1.554507e+05,1346.643975
std,1.996485,9.893847e+10,116.511888,101.804938,4.444187e+05,14003.313610,1.041636e+05,6.689215e+04,2.514138e+05,1.495312e+05,1.308613e+05,411.039689,333.905884,3.901120e+05,6164.552399
min,2006.000000,1.000100e+04,1.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000
25%,2006.000000,1.055500e+04,4.000000,3.000000,2.829208e+03,625.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000,5.000000,2.689600e+03,352.500000
50%,2010.000000,5.000000e+10,9.000000,8.000000,1.334511e+04,1537.775000,1.125000e+03,1.350000e+03,0.000000e+00,1.121600e+03,0.000000e+00,20.000000,18.000000,1.226735e+04,711.365000
75%,2010.000000,1.900000e+11,23.000000,21.000000,1.046682e+05,4666.247500,6.196375e+03,1.574625e+04,1.725875e+04,9.600000e+03,0.000000e+00,80.000000,68.000000,9.633950e+04,1399.812500
max,2010.000000,2.700000e+11,6997.000000,5800.000000,5.690642e+06,500180.000000,3.664205e+06,1.403049e+06,3.178226e+06,5.525600e+06,2.745700e+06,9932.000000,8359.000000,4.989491e+06,500177.000000


Ainda podemos ver que o conjunto de dados conta com $7622$ entradas, mas algumas colunas apresentam valores não informados:

In [157]:
train.count()

ano                                      7622
sequencial_candidato                     7622
nome                                     7622
uf                                       7622
partido                                  7622
quantidade_doacoes                       7622
quantidade_doadores                      7622
total_receita                            7622
media_receita                            7622
recursos_de_outros_candidatos.comites    7622
recursos_de_pessoas_fisicas              7622
recursos_de_pessoas_juridicas            7622
recursos_proprios                        7622
recursos_de_partido_politico             7622
quantidade_despesas                      7622
quantidade_fornecedores                  7622
total_despesa                            7622
media_despesa                            7622
cargo                                    7622
sexo                                     7622
grau                                     7622
estado_civil                      

### 2. Análise de balanceamento das classes

Após conhecer os dados, queremos conhecer as variáveis que apresentam algum tipo de viés e realizar as devidas transformações, preencher valores não informados que sejam de interesse e retirar dados que não ajudem nas nossas predições. 

#### 2.1. Lidando com valores não informados

Inicialmente, checamos se existem dados faltando no conjunto de treino, e verificamos que todas as entradas possuem todos os atributos preenchidos:

In [158]:
train.isnull().sum()

ano                                      0
sequencial_candidato                     0
nome                                     0
uf                                       0
partido                                  0
quantidade_doacoes                       0
quantidade_doadores                      0
total_receita                            0
media_receita                            0
recursos_de_outros_candidatos.comites    0
recursos_de_pessoas_fisicas              0
recursos_de_pessoas_juridicas            0
recursos_proprios                        0
recursos_de_partido_politico             0
quantidade_despesas                      0
quantidade_fornecedores                  0
total_despesa                            0
media_despesa                            0
cargo                                    0
sexo                                     0
grau                                     0
estado_civil                             0
ocupacao                                 0
situacao   

#### 2.2. Balanceamento de classes

Queremos ver o balanceamento entre as classes definidas pela variável ```situacao```, que indica se o candidato foi eleito ou não nos anos de 2006 e 2010. Para isso, vamos ver quantas entradas constam como ```ELEITO``` e ```NAO_ELEITO```:

In [159]:
train["situacao"].value_counts()

nao_eleito    6596
eleito        1026
Name: situacao, dtype: int64

O que em proporção seria:

In [160]:
train["situacao"].value_counts(normalize = True)

nao_eleito    0.86539
eleito        0.13461
Name: situacao, dtype: float64

Ou seja, aproximadamente ```86%``` das entradas constam como não eleitos, e ```13%``` eleitos. Em proporção, para cada candidato marcado como eleito, aproximadamente 7 são nao_eleitos. Claramente vemos que há um desbalanceamento entre as classes. Isso pode causar efeitos colaterais na predição como decisões enviesadas, overfitting, e de maneira mais prática, pode significar que o classificador vai acertar mais para o lado com maior numero de ocorrências, o que no nosso caso seria refletido em um número maior de acertos sobre quem não vai se eleger do que sobre quem de fato seria eleito. 

Para resolver esse problema podemos fazer uso de validação cruzada, utilização de parâmetros que possam ser tunados nos modelos, bem como técnicas de *downsampling*, significando uma diminuição da amostragem da classe majoritária, ou *upsampling*, o aumento da amostragem da classe minoritária.

#### 2.3. Tratando variáveis

Vamos começar eliminando variáveis que não queremos considerar no treinamento do modelo. Por exemplo, o nome do candidato não deve influenciar sua eleição, seu número, etc. Escolhemos eliminar as que seguem:

In [161]:
train.drop(['nome'], axis=1, inplace=True)
train.drop(['uf'], axis=1, inplace=True)
train.drop(['partido'], axis=1, inplace=True)
train.drop(['cargo'], axis=1, inplace=True)
train.drop(['grau'], axis=1, inplace=True)
train.drop(['estado_civil'], axis=1, inplace=True)
train.drop(['sequencial_candidato'], axis=1, inplace=True)
train.drop(['ano'], axis=1, inplace=True)

Agora vamos transformar variáveis categóricas em binários, especialmente a variável de interesse, ```situacao```, ```sexo```, e ```cargo```, que consideramos poder influenciar a eleição ou não de um candidato:

In [162]:
train = pd.get_dummies(train)
train.describe()

,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,...,ocupacao_TÉCNICO EM EDIFICAÇÕES,ocupacao_TÉCNICO EM INFORMÁTICA,ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",ocupacao_VEREADOR,ocupacao_VETERINÁRIO,ocupacao_VIGILANTE,ocupacao_ZOOTECNISTA,situacao_eleito,situacao_nao_eleito
count,7622.000000,7622.000000,7.622000e+03,7622.000000,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7622.000000,...,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000,7622.000000
mean,27.673839,25.239963,1.732199e+05,5456.131027,1.965750e+04,2.315820e+04,7.921689e+04,2.577600e+04,2.541136e+04,130.485174,...,0.000131,0.000918,0.003149,0.004198,0.043165,0.002230,0.002755,0.000131,0.134610,0.865390
std,116.511888,101.804938,4.444187e+05,14003.313610,1.041636e+05,6.689215e+04,2.514138e+05,1.495312e+05,1.308613e+05,411.039689,...,0.011454,0.030293,0.056029,0.064663,0.203241,0.047177,0.052421,0.011454,0.341329,0.341329
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,3.000000,2.829208e+03,625.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,9.000000,8.000000,1.334511e+04,1537.775000,1.125000e+03,1.350000e+03,0.000000e+00,1.121600e+03,0.000000e+00,20.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,23.000000,21.000000,1.046682e+05,4666.247500,6.196375e+03,1.574625e+04,1.725875e+04,9.600000e+03,0.000000e+00,80.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,6997.000000,5800.000000,5.690642e+06,500180.000000,3.664205e+06,1.403049e+06,3.178226e+06,5.525600e+06,2.745700e+06,9932.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Das variáveis resultantes nessa transformação, vamos considerar apenas as seguintes:

In [163]:
train_vars = ['quantidade_doacoes', 'quantidade_doadores', 'total_receita', 'media_receita', 'quantidade_despesas', 'total_despesa', 'media_despesa', 'sexo_MASCULINO', 'ocupacao_VEREADOR', 'situacao_eleito']
train = train[train_vars]
train.describe()

,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,quantidade_despesas,total_despesa,media_despesa,sexo_MASCULINO,ocupacao_VEREADOR,situacao_eleito
count,7622.000000,7622.000000,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000,7622.000000,7622.000000,7622.000000
mean,27.673839,25.239963,1.732199e+05,5456.131027,130.485174,1.554507e+05,1346.643975,0.858567,0.043165,0.134610
std,116.511888,101.804938,4.444187e+05,14003.313610,411.039689,3.901120e+05,6164.552399,0.348490,0.203241,0.341329
min,1.000000,1.000000,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,4.000000,3.000000,2.829208e+03,625.000000,6.000000,2.689600e+03,352.500000,1.000000,0.000000,0.000000
50%,9.000000,8.000000,1.334511e+04,1537.775000,20.000000,1.226735e+04,711.365000,1.000000,0.000000,0.000000
75%,23.000000,21.000000,1.046682e+05,4666.247500,80.000000,9.633950e+04,1399.812500,1.000000,0.000000,0.000000
max,6997.000000,5800.000000,5.690642e+06,500180.000000,9932.000000,4.989491e+06,500177.000000,1.000000,1.000000,1.000000


#### 2.4. Corrigindo desbalanceamento usando SMOTE

Vamos usar uma técnica de *oversampling* chamada SMOTE. Baseado em uma implementação de seu algoritmo, teremos: 

In [164]:
X = train.loc[:, train.columns != 'situacao_eleito']
y = train.loc[:, train.columns == 'situacao_eleito']

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y = pd.DataFrame(data=os_data_y.ravel(),columns=['situacao_eleito'])

#checando o novo balanceamento

print("Tamanho do dado oversampled é ",len(os_data_X))
print("Número de candidados não eleitos nos dados oversampled: ",len(os_data_y[os_data_y['situacao_eleito']==0]))
print("Número de candidatos eleitos: ",len(os_data_y[os_data_y['situacao_eleito']==1]))
print("Proporção de candidatos não eleitos no dado oversampled: ",len(os_data_y[os_data_y['situacao_eleito']==0])/len(os_data_X))
print("Proporção de candidatos eleitos no dado oversampled: ",len(os_data_y[os_data_y['situacao_eleito']==1])/len(os_data_X))

Tamanho do dado oversampled é  9274
Número de candidados não eleitos nos dados oversampled:  4637
Número de candidatos eleitos:  4637
Proporção de candidatos não eleitos no dado oversampled:  0.5
Proporção de candidatos eleitos no dado oversampled:  0.5


### 3. Treinando modelos

Vamos treinar alguns modelos de classificação para predizer quem foi eleito ou não na eleição de 2014. Primeiro definimos uma função que calcule o RMSE para os modelos usando validação cruzada. Essa função será usada para avaliação de cada modelo:

In [165]:
def rmse_cv(model, matrix, target):
    rmse = np.sqrt(-cross_val_score(model, matrix, target, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

#### 3.1. Regressão logística

Inicialmente vamos construir o modelo:

In [166]:
from sklearn.linear_model import LogisticRegression
classifier_logistic = LogisticRegression(solver='lbfgs')
classifier_logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

Em seguida, calculamos a confusion matrix, que nos diz que temos 1897 + 150 predições corretas, e 178 + 62 predições possivelmente incorretas:

In [167]:
y_pred_logistic = classifier_logistic.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_logistic)
print(confusion_matrix)

[[1897   62]
 [ 178  150]]


A acurácia do classificador pode ser calculada como:

In [168]:
print('Acurácia do classificador com regresssão logística é: {:.2f}'.format(classifier_logistic.score(X_test, y_test)))

Acurácia do classificador com regresssão logística é: 0.90


E finalmente, considerando ```0 = não_eleito``` e ```1 = eleito```, temos o cálculo de Precision, Recall e AUC-Precision&Recall para o modelo com regressão logística:

In [169]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_logistic))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1959
           1       0.71      0.46      0.56       328

   micro avg       0.90      0.90      0.90      2287
   macro avg       0.81      0.71      0.75      2287
weighted avg       0.88      0.90      0.89      2287



O modelo apresenta o menor rmse como sendo:

In [170]:
min(rmse_cv(classifier_logistic, X_train, y_train))

0.31069652982570584

In [171]:
min(rmse_cv(classifier_logistic, X_test, y_test))

0.299853765528723

#### 3.2. KNN

Usando KNN queremos encontrar um valor de K que ache o menor RMSE para as nossas predições. O modelo segue abaixo e retorna o menor RMSE para K variando de $0$ a $100$:

In [172]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

from math import sqrt

def calculate_knn(matrix, target):
    rmse_val = [] #to store rmse values for different k
    
    for K in range(100):
        K = K+1
        model = KNeighborsRegressor(n_neighbors = K)
        model.fit(matrix, target)  #fit the model

        errors = rmse_cv(model, matrix, target)
        rmse_val.append(errors.mean()) #store rmse values
    
    print('O menor valor de RMSE é', min(rmse_val), 'para K =', rmse_val.index(min(rmse_val)))

In [173]:
calculate_knn(X_train, y_train)

O menor valor de RMSE é 0.24313784652236548 para K = 79


Em seguida, usamos esse valor de K para criar o modelo e aplicá-lo:

In [174]:
classifier_knn = KNeighborsRegressor(n_neighbors = 79)
classifier_knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=79, p=2,
          weights='uniform')

A predição desse modelo gera valores diferentes de 0 e 1, portanto, vamos tratá-lo para considerar valores menores ou iguais a $0.5$ como sendo 0, e os demais 1:

In [175]:
y_pred_knn = classifier_knn.predict(X_test)
print(y_pred_knn)

[[0.25316456]
 [0.        ]
 [0.        ]
 ...
 [0.67088608]
 [0.        ]
 [0.        ]]


In [176]:
y_pred_knn_format = []
for i in y_pred_knn:
    if i <= 0.5:
        y_pred_knn_format.append(0)
    else:
        y_pred_knn_format.append(1)

Isso gera uma matriz de predição como a que segue:

In [177]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_knn_format)
print(confusion_matrix)

[[1870   89]
 [ 117  211]]


E uma acurácia para o modelo KNN de:

In [178]:
print('Acurácia do classificador com KNN é: {:.2f}'.format(classifier_knn.score(X_test, y_test)))

Acurácia do classificador com KNN é: 0.49


Por fim, a avaliação do classificador fica como segue:

In [179]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_knn_format))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1959
           1       0.70      0.64      0.67       328

   micro avg       0.91      0.91      0.91      2287
   macro avg       0.82      0.80      0.81      2287
weighted avg       0.91      0.91      0.91      2287



#### 3.3. Árvore de decisão

Vamos gerar predições com duas configurações de árvore de decisão. A primeira, com critério ```gini```, e a segunda com critério ```entropy```:

In [180]:
from sklearn.tree import DecisionTreeClassifier

classifier_dt_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, max_depth=3, min_samples_leaf=5)
classifier_dt_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [181]:
classifier_dt_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100, max_depth=3, min_samples_leaf=5)
classifier_dt_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

Em seguida, calculamos a predição para ```gini```:

In [182]:
y_pred_dt_gini = classifier_dt_gini.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_dt_gini)
print(confusion_matrix)

[[1906   53]
 [ 170  158]]


E para ```entropy```:

In [183]:
y_pred_dt_entropy = classifier_dt_entropy.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_dt_entropy)
print(confusion_matrix)

[[1892   67]
 [ 131  197]]


Para os valores de acurácia, ficamos como segue, o critério ```entropy``` apresentando uma acurácia um pouco maior:

In [184]:
print('Acurácia do classificador com Árvore de Decisão com critério gini é: {:.2f}'.format(classifier_dt_gini.score(X_test, y_test)))

Acurácia do classificador com Árvore de Decisão com critério gini é: 0.90


In [185]:
print('Acurácia do classificador com Árvore de Decisão com critério entropy é: {:.2f}'.format(classifier_dt_entropy.score(X_test, y_test)))

Acurácia do classificador com Árvore de Decisão com critério entropy é: 0.91


Podemos ver um report dessa classifcação abaixo, primeiro para ```gini``` e em seguida para ```entropy```:

In [186]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_dt_gini))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1959
           1       0.75      0.48      0.59       328

   micro avg       0.90      0.90      0.90      2287
   macro avg       0.83      0.73      0.77      2287
weighted avg       0.89      0.90      0.89      2287



In [187]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_dt_entropy))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1959
           1       0.75      0.60      0.67       328

   micro avg       0.91      0.91      0.91      2287
   macro avg       0.84      0.78      0.81      2287
weighted avg       0.91      0.91      0.91      2287



E finalmente, os valores de RMSE com validação cruzada para ```gini```:

In [188]:
min(rmse_cv(classifier_dt_gini, X_train, y_train))

0.3029196256855408

In [189]:
min(rmse_cv(classifier_dt_gini, X_test, y_test))

0.2848514540549038

E para ```entropy```:

In [190]:
min(rmse_cv(classifier_dt_entropy, X_train, y_train))

0.29509092104873924

In [191]:
min(rmse_cv(classifier_dt_entropy, X_test, y_test))

0.2730593406363751

#### 3.4. Adaboost

Utilizando agora um modelo Adaboost, temos:

In [192]:
from sklearn.ensemble import AdaBoostClassifier

classifier_adaboost = AdaBoostClassifier(n_estimators=50, learning_rate=1, random_state=0)
classifier_adaboost.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=50, random_state=0)

Em seguida calculamos a predição:

In [193]:
y_pred_adaboost = classifier_adaboost.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred_adaboost)
print(confusion_matrix)

[[1860   99]
 [ 126  202]]


E sua acurácia:

In [194]:
print('Acurácia do classificador com Adaboost é: {:.2f}'.format(classifier_adaboost.score(X_test, y_test)))

Acurácia do classificador com Adaboost é: 0.90


Um report da classifcação segue abaixo:

In [195]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_adaboost))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1959
           1       0.67      0.62      0.64       328

   micro avg       0.90      0.90      0.90      2287
   macro avg       0.80      0.78      0.79      2287
weighted avg       0.90      0.90      0.90      2287



E finalmente o valor do RMSE:

In [196]:
min(rmse_cv(classifier_adaboost, X_train, y_train))

0.29509092104873924

In [197]:
min(rmse_cv(classifier_adaboost, X_test, y_test))

0.2880441499395717

### 4. Avaliação baseada em acurácia e precisão

Em termos de acurácia, o modelo que mostrou melhor desempenho foi o Decision Tree, com ```0.91``` e o KNN com pior acurácia, sendo esta de ```0.49```. Em termos de precisão, os modelos tiveram desempenho similar baseado nos reports obtidos. Para os eleitos, a precisão variou de ```0.67``` a ```0.75```, sendo a maior precisão também Decision Tree. O erro por sua vez foi menor para o KNN, com ```0.24```.

Considerando isso, vamos pegar os valores de precisão para Decision Tree e submetê-los ao Kaggle.

In [204]:
test = pd.read_csv("test.csv")
test = pd.get_dummies(test)
test_vars = ['quantidade_doacoes', 'quantidade_doadores', 'total_receita', 'media_receita', 'quantidade_despesas', 'total_despesa', 'media_despesa', 'sexo_MASCULINO', 'ocupacao_VEREADOR']
ids = test["sequencial_candidato"]
test = test[test_vars]

In [215]:
import csv 

def situacao_str(situacao):
    if situacao == 1:
        return "eleito"
    return "nao_eleito"

resultado = {"Id": ids, "Predicted": list(map(situacao_str, classifier_dt_entropy.predict(test)))}
pd.DataFrame(resultado).to_csv("saida.csv", quotechar="\"", quoting=csv.QUOTE_NONNUMERIC, index=False)